In [1]:
import sys

!{sys.executable} -m pip install torch numpy transformers pandas scikit-learn sklearn

     |████████████████████████████████| 3.4 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 39.2 MB/s 
     |████████████████████████████████| 61 kB 266 kB/s 
     |████████████████████████████████| 895 kB 38.7 MB/s 
     |████████████████████████████████| 596 kB 39.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [16]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1IQcFO7AK44MD06auLzL8DeURNyYdoZUf' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1IQcFO7AK44MD06auLzL8DeURNyYdoZUf" -O test_dataset.csv && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1U1CWrwMO8sK_iSRebG1Ueps2At2wmH3z' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1U1CWrwMO8sK_iSRebG1Ueps2At2wmH3z" -O train_dataset.csv && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MBDkVY-mID6RAT_xeMB-aEGG1rmajjPJ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1MBDkVY-mID6RAT_xeMB-aEGG1rmajjPJ" -O model_200000.pt && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=11zuBIqA1_m81Jef17OECEhe3nprpN14R' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=11zuBIqA1_m81Jef17OECEhe3nprpN14R" -O sarcasm_detection_model.pt && rm -rf /tmp/cookies.txt


--2021-12-19 17:27:34--  https://docs.google.com/uc?export=download&confirm=&id=1IQcFO7AK44MD06auLzL8DeURNyYdoZUf
Resolving docs.google.com (docs.google.com)... 142.251.8.138, 142.251.8.101, 142.251.8.100, ...
Connecting to docs.google.com (docs.google.com)|142.251.8.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-78-docs.googleusercontent.com/docs/securesc/56qbeq0l58ei5f720bu7f0onb7r6jqmh/30f20red5m1f4boegdrqg1pfmtu52cmb/1639934850000/15627403865753104419/05393095360605391833Z/1IQcFO7AK44MD06auLzL8DeURNyYdoZUf?e=download [following]
--2021-12-19 17:27:35--  https://doc-08-78-docs.googleusercontent.com/docs/securesc/56qbeq0l58ei5f720bu7f0onb7r6jqmh/30f20red5m1f4boegdrqg1pfmtu52cmb/1639934850000/15627403865753104419/05393095360605391833Z/1IQcFO7AK44MD06auLzL8DeURNyYdoZUf?e=download
Resolving doc-08-78-docs.googleusercontent.com (doc-08-78-docs.googleusercontent.com)... 142.251.8.132, 2404:6800:4008:c15::84
Connecting to doc-0

In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel
import numpy as np

MODEL = "cardiffnlp/twitter-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

roberta = AutoModel.from_pretrained(MODEL).to(device)


Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split


def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


df = pd.read_csv("train_dataset.csv")
X = df["tweet_content"].tolist()
X = [preprocess(x) for x in X]
y = (df["sarcasm_label"] == 'sarcastic').tolist()


In [6]:
from torch.nn.functional import pad

X_encoded = tokenizer(X, return_tensors='pt', padding=True)['input_ids']

In [7]:
X_encoded.shape

torch.Size([2825, 127])

In [8]:
X_train, X_cross, y_train, y_cross = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
y_train = torch.Tensor(y_train).type(torch.long)
y_cross = torch.Tensor(y_cross).type(torch.long)


In [9]:
import torch
from torch import nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = roberta
        self.linear1 = nn.Linear(4608, 800)
        self.linear2 = nn.Linear(800, 20)
        self.linear3 = nn.Linear(20, 2)
        self.pooling = nn.MaxPool2d((20, 1))
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = self.roberta(x).last_hidden_state
        x = self.pooling(x)
        x = x.reshape(x.shape[0], -1)
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear3(x)
        return x

In [10]:
from torch.utils.data import TensorDataset, DataLoader

dataset_train = TensorDataset(X_train, y_train)  # create your datset
dataloader_train = DataLoader(dataset_train, batch_size=40)  # create your dataloader
dataset_cross = TensorDataset(X_cross, y_cross)  # create your datset
dataloader_cross = DataLoader(dataset_cross, batch_size=40)  # create your dataloader

In [11]:
def train(dataloader, model, loss_fn, optimizer):  #fragment z dokumentacji pytorch
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss :>7f}  [{current:>5d}/{size:>5d}]")
    train_loss /= num_batches
    print(f"Avg train loss: {train_loss:>8f} \n")


In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    true_positive, true_negative, false_negative, false_positive = 0.0001, 0.0001, 0.0001, 0.0001  #zeby dzielenia przez zero nie bylo
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            y_pred = pred.argmax(1)
            correct += (y_pred == y).type(torch.long).sum().item()

            true_positive += torch.logical_and(y_pred == y, y == 1).sum().item()
            true_negative += torch.logical_and(y_pred == y, y == 0).sum().item()
            false_negative += torch.logical_and(y_pred != y, y == 1).sum().item()
            false_positive += torch.logical_and(y_pred != y, y == 0).sum().item()
    recall = true_positive / (true_positive + false_negative)
    precision = true_positive / (true_positive + false_positive)
    specificity = true_negative / (false_positive + true_negative)
    f_score = (2 * precision * recall) / (precision + recall)
    g_mean = (recall * specificity) ** (0.5)

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(
        f"\n recall: {recall} \n precision: {precision} \n specificity: {specificity} \n f_score: {f_score} \n g_mean: {g_mean}\n")

In [17]:
model = Model().to(device)
# pretraining na reddicie
#model.load_state_dict(torch.load("model_200000.pt"))
# pretraining na reddicie + trening na isarcasm
model.load_state_dict(torch.load("sarcasm_detection_model.pt"))

<All keys matched successfully>

In [18]:
loss_fn = nn.CrossEntropyLoss(weight=torch.Tensor([1.0, 1.0])).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [25]:
epochs = 11
for t in range(epochs):
    train(dataloader_train, model, loss_fn, optimizer)
    test(dataloader_cross, model, loss_fn)
print("Done!")

loss: 0.055963  [    0/ 2260]


KeyboardInterrupt: ignored

In [19]:
test(dataloader_cross, model, loss_fn)

Test Error: 
 Accuracy: 81.2%, Avg loss: 0.941372 


 recall: 0.5833331349211073 
 precision: 0.4083334861108565 
 specificity: 0.8523907058666504 
 f_score: 0.480392195309421 
 g_mean: 0.7051437744395171



In [33]:
df_test = pd.read_csv("test_dataset.csv")
X_test = df_test["tweet_content"].tolist()
X_test = [preprocess(x) for x in X_test]
y_test = (df_test["sarcasm_label"] == 'sarcastic').tolist()

In [34]:
X_encoded_test = tokenizer(X_test, return_tensors='pt', padding=True)['input_ids']
X_encoded_test = X_encoded_test[:,:127]
y_test = torch.Tensor(y_test).type(torch.long)

In [35]:
dataset_test = TensorDataset(X_encoded_test, y_test)
dataloader_test = DataLoader(dataset_test, batch_size=40)

In [36]:
test(dataloader_test, model, loss_fn)

Test Error: 
 Accuracy: 81.0%, Avg loss: 0.709910 


 recall: 0.6134451874870799 
 precision: 0.4506173449168581 
 specificity: 0.849407665175071 
 f_score: 0.5195729258748386 
 g_mean: 0.7218483527835221



In [ ]:
model_name = "sarcasm_detection_model.pt"
torch.save(model.state_dict(), model_name)